
**Community edition of CPLEX- Step 1: Installation**














In [ ]:
!pip install docplex
!pip install cplex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.5/646.5 kB 10.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for docplex: filename=docplex-2.30.251-py3-none-any.whl size=685954 sha256=b46b908b386add8aced22b9294c6dcfdb80357a5032abb34cefc36ee8397f0a4
  Stored in directory: /root/.cache/pip/wheels/c1/d9/5c/1d919b1e441ebbaff244513a13d09c3c0378401601d4184bb0
Successfully built docplex
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 20.0 MB/s eta 0:00:00


**Step 2: Load JSON in Colab**

In [ ]:
import json

with open("test.json", "r") as f:
    data = json.load(f)

# Extract services, nodes, intentions
services = {s["id"]: s["resources"] for s in data["services"]}
nodes = {n["id"]: {**n["capacity"], "L": data["latency"][n["id"]][0]} for n in data["nodes"]}
intentions = {i["id"]: {"services": i["services"], "QoS": i["QoS"], "weight": i["weight"]} for i in data["intentions"]}
resources = ["CPU","MEM","DISK","BW"]

print("✅ Data loaded successfully")
print("Services:", services.keys())
print("Nodes:", nodes.keys())
print("Intentions:", intentions.keys())


✅ Data loaded successfully
Services: dict_keys(['s1', 's2', 's3', 's4'])
Nodes: dict_keys(['g1', 'g2', 'n1', 'n2', 'n3', 'n4'])
Intentions: dict_keys(['i1', 'i2', 'i3', 'i4'])


**Step 3: Build DOcplex Model**

In [ ]:
from docplex.mp.model import Model
import time   # for execution time

# Build Model
mdl = Model("multi_intention_service_placement")

# Decision variables: x[s,n] = 1 if service s is placed on node n
x = {(s,n): mdl.binary_var(name=f"x_{s}_{n}") for s in services for n in nodes}

# Objective: Minimize weighted latency
mdl.minimize(
    mdl.sum(intentions[i]["weight"] *
            mdl.sum(nodes[n]["L"]*x[s,n] for s in intentions[i]["services"] for n in nodes)
            for i in intentions)
)

# Constraint: Each service placed exactly once
for s in services:
    mdl.add_constraint(mdl.sum(x[s,n] for n in nodes) == 1, ctname=f"place_{s}")

# Constraint: Resource capacity per node
for n in nodes:
    for r in resources:
        mdl.add_constraint(mdl.sum(services[s][r]*x[s,n] for s in services) <= nodes[n][r],
                           ctname=f"cap_{n}_{r}")

# Constraint: QoS latency + bandwidth per intention
for i in intentions:
    mdl.add_constraint(
        mdl.sum(nodes[n]["L"]*x[s,n] for s in intentions[i]["services"] for n in nodes)
        <= intentions[i]["QoS"]["latency"], ctname=f"lat_{i}"
    )
    mdl.add_constraint(
        mdl.sum(services[s]["BW"]*x[s,n] for s in intentions[i]["services"] for n in nodes)
        >= intentions[i]["QoS"]["bandwidth"], ctname=f"bw_{i}"
    )

# --- Solve with computing the execution time --- To be compared with the LLM reasoning Agent
start_time = time.time()
solution = mdl.solve(log_output=True)
end_time = time.time()
execution_time = end_time - start_time

# Results
if solution:
    print("Optimal solution found")
    print("Objective value:", solution.objective_value)
    print("Execution time:", round(execution_time, 4), "seconds")
    for (s,n), var in x.items():
        if var.solution_value > 0.5:
            print("Service", s, "placed on node", n)
else:
    print("No feasible solution found")
    print("Execution time:", round(execution_time, 4), "seconds")


Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 36 rows and 24 columns.
MIP Presolve modified 3 coefficients.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.03 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.03 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.03 ticks)
Optimal solution found
Objective value: 2180.0
Execution time: 0.0144 seconds
Service s1 placed on node g1
Service s2 placed on node g2
Service s3 placed on node n4
Service s4 placed on node n2


**Testing with the node -intention latency-**

In [ ]:
from docplex.mp.model import Model
import time

# =========================
# Build Optimization Model
# =========================
mdl = Model("multi_intention_service_placement")

# Decision variables
x = {(s,n): mdl.binary_var(name=f"x_{s}_{n}") for s in services for n in nodes}
y = {(i,n): mdl.binary_var(name=f"y_{i}_{n}") for i in intentions for n in nodes}

# Objective: minimize weighted latency (node-based, not service-based)
mdl.minimize(
    mdl.sum(intentions[i]["weight"] *
            mdl.sum(nodes[n]["L"] * y[i,n] for n in nodes)
            for i in intentions)
)

# -------------------------
# Constraints
# -------------------------

# 1. Unique placement of each service
for s in services:
    mdl.add_constraint(mdl.sum(x[s,n] for n in nodes) == 1, ctname=f"place_{s}")

# 2. Link x and y: if a service of i is on node n, then intention i uses node n
for i in intentions:
    for s in intentions[i]["services"]:
        for n in nodes:
            mdl.add_constraint(y[i,n] >= x[s,n], ctname=f"link_{i}_{s}_{n}")

# 3. Node capacity (services only)
for n in nodes:
    for r in resources:
        mdl.add_constraint(
            mdl.sum(services[s][r] * x[s,n] for s in services) <= nodes[n][r],
            ctname=f"cap_{n}_{r}"
        )

# 4. QoS constraints
for i in intentions:
    # Latency (per node, not per service)
    mdl.add_constraint(
        mdl.sum(nodes[n]["L"] * y[i,n] for n in nodes) <= intentions[i]["QoS"]["latency"],
        ctname=f"lat_{i}"
    )
    # Bandwidth (per service, aggregated)
    mdl.add_constraint(
        mdl.sum(services[s]["BW"] * x[s,n] for s in intentions[i]["services"] for n in nodes)
        >= intentions[i]["QoS"]["bandwidth"], ctname=f"bw_{i}"
    )

# 5. Extra resources constraints (only if defined in JSON)
for i in intentions:
    if "extra_resources" in intentions[i]:
        for r, val in intentions[i]["extra_resources"].items():
            mdl.add_constraint(
                mdl.sum(services[s][r] * x[s,n] for s in intentions[i]["services"] for n in nodes)
                + val <= max(nodes[n][r] for n in nodes),
                ctname=f"extra_{i}_{r}"
            )

# =========================
# Solve
# =========================
start_time = time.time()
solution = mdl.solve(log_output=True)
end_time = time.time()
execution_time = end_time - start_time

# =========================
# Results
# =========================
if solution:
    print("\nOptimal solution found")
    print("Objective value:", solution.objective_value)
    print("Execution time:", round(execution_time, 4), "seconds")
    print("\nService placement:")
    for (s,n), var in x.items():
        if var.solution_value > 0.5:
            print(f" - Service {s} → Node {n}")
    print("\nNode usage per intention:")
    for (i,n), var in y.items():
        if var.solution_value > 0.5:
            print(f" - Intention {i} uses Node {n}")
else:
    print("\nNo feasible solution found")
    print("Execution time:", round(execution_time, 4), "seconds")


Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 72 rows and 48 columns.
MIP Presolve modified 9 coefficients.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.04 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.05 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.05 ticks)

Optimal solution found
Objective value: 2180.0
Execution time: 0.0179 seconds

Service placement:
 - Service s1 → Node g1
 - Service s2 → Node g2
 - Service s3 → Node n4
 - Service s4 → Node n2

Node usage per intention:
 - Intention i1 uses Node g2
 - Intention i2 uses Node g2
 - Intention i2 uses Node n4
 - Intention i3 uses Node n4
 - Intention i4 uses Node g2
 - Intentio

**Now Testing with more data in the Json**

**To be continued with an advanced dataset**

In [ ]:
import json

with open("test2.json", "r") as f:
    data = json.load(f)

# Extract services, nodes, intentions
services = {s["id"]: s["resources"] for s in data["services"]}
nodes = {n["id"]: {**n["capacity"], "L": data["latency"][n["id"]][0]} for n in data["nodes"]}
intentions = {i["id"]: {"services": i["services"], "QoS": i["QoS"], "weight": i["weight"], "extra_resources": i.get("extra_resources", {})} for i in data["intentions"]}
resources = ["CPU","MEM","DISK","BW"]

print("✅ Data loaded successfully")
print("Services:", services.keys())
print("Nodes:", nodes.keys())
print("Intentions:", intentions.keys())

✅ Data loaded successfully
Services: dict_keys(['s1', 's2', 's3', 's4', 's5', 's6'])
Nodes: dict_keys(['g1', 'g2', 'n1', 'n2', 'n3'])
Intentions: dict_keys(['i1', 'i2', 'i3', 'i4', 'i5', 'i6'])


In [ ]:
print("Node capacities:")
for n, cap in nodes.items():
    print(n, cap)


Node capacities:
g1 {'CPU': 2, 'MEM': 4, 'DISK': 2, 'BW': 100, 'L': 30}
g2 {'CPU': 2, 'MEM': 3, 'DISK': 1, 'BW': 80, 'L': 35}
n1 {'CPU': 4, 'MEM': 10, 'DISK': 10, 'BW': 120, 'L': 60}
n2 {'CPU': 8, 'MEM': 16, 'DISK': 20, 'BW': 200, 'L': 45}
n3 {'CPU': 10, 'MEM': 32, 'DISK': 30, 'BW': 250, 'L': 55}


In [ ]:
print("Resources per service:")
for s, res in services.items():
    print(s, res)


Resources per service:
s1 {'CPU': 1, 'MEM': 2, 'DISK': 1, 'BW': 60}
s2 {'CPU': 2, 'MEM': 4, 'DISK': 1, 'BW': 80}
s3 {'CPU': 6, 'MEM': 12, 'DISK': 4, 'BW': 120}
s4 {'CPU': 2, 'MEM': 6, 'DISK': 2, 'BW': 100}
s5 {'CPU': 3, 'MEM': 8, 'DISK': 2, 'BW': 40}
s6 {'CPU': 5, 'MEM': 10, 'DISK': 3, 'BW': 90}


In [ ]:
print("Checking capacities used for constraints:")
for n in nodes:
    for r in resources:
        print(n, r, "capacity =", nodes[n][r])


Checking capacities used for constraints:
g1 CPU capacity = 2
g1 MEM capacity = 4
g1 DISK capacity = 2
g1 BW capacity = 100
g2 CPU capacity = 2
g2 MEM capacity = 3
g2 DISK capacity = 1
g2 BW capacity = 80
n1 CPU capacity = 4
n1 MEM capacity = 10
n1 DISK capacity = 10
n1 BW capacity = 120
n2 CPU capacity = 8
n2 MEM capacity = 16
n2 DISK capacity = 20
n2 BW capacity = 200
n3 CPU capacity = 10
n3 MEM capacity = 32
n3 DISK capacity = 30
n3 BW capacity = 250


In [ ]:
print("Model constraints overview:")
mdl.print_information()


Model constraints overview:
Model: multi_intention_service_placement
 - number of variables: 30
   - binary=30, integer=0, continuous=0
 - number of constraints: 38
   - linear=38
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [ ]:
from docplex.mp.model import Model
import time

# =========================
# Build Optimization Model
# =========================
mdl = Model("multi_intention_service_placement")

# Decision vars: x[s,n] = 1 if service s placed on node n
x = {(s,n): mdl.binary_var(name=f"x_{s}_{n}") for s in services for n in nodes}

# Objective: minimize weighted latency (sum version)
mdl.minimize(
    mdl.sum(intentions[i]["weight"] *
            mdl.sum(nodes[n]["L"] * x[s,n] for s in intentions[i]["services"] for n in nodes)
            for i in intentions)
)

# -------------------------
# Constraints
# -------------------------

# 1. Unique placement of each service
for s in services:
    mdl.add_constraint(mdl.sum(x[s,n] for n in nodes) == 1, ctname=f"place_{s}")

# 2. Node capacity (services only)
for n in nodes:
    for r in resources:
        mdl.add_constraint(
            mdl.sum(services[s][r] * x[s,n] for s in services) <= nodes[n][r],
            ctname=f"cap_{n}_{r}"
        )

# 3. QoS latency + bandwidth constraints
for i in intentions:
    # Latency constraint (sum of latencies of all services in intention i)
    mdl.add_constraint(
        mdl.sum(nodes[n]["L"] * x[s,n] for s in intentions[i]["services"] for n in nodes)
        <= intentions[i]["QoS"]["latency"], ctname=f"lat_{i}"
    )
    # Bandwidth constraint
    mdl.add_constraint(
        mdl.sum(services[s]["BW"] * x[s,n] for s in intentions[i]["services"] for n in nodes)
        >= intentions[i]["QoS"]["bandwidth"], ctname=f"bw_{i}"
    )

# 4. Extra resources constraints (only if defined in JSON)
for i in intentions:
    if "extra_resources" in intentions[i]:
        for r, val in intentions[i]["extra_resources"].items():
            mdl.add_constraint(
                mdl.sum(services[s][r] * x[s,n] for s in intentions[i]["services"] for n in nodes)
                + val <= max(nodes[n][r] for n in nodes),
                ctname=f"extra_{i}_{r}"
            )

# =========================
# Solve
# =========================
start_time = time.time()
solution = mdl.solve(log_output=True)
end_time = time.time()
execution_time = end_time - start_time

# =========================
# Results
# =========================
if solution:
    print("\nOptimal solution found")
    print("Objective value:", solution.objective_value)
    print("Execution time:", round(execution_time, 4), "seconds")
    print("\nService placement:")
    for (s,n), var in x.items():
        if var.solution_value > 0.5:
            print(f" - Service {s} → Node {n}")
else:
    print("\nNo feasible solution found")
    print("Execution time:", round(execution_time, 4), "seconds")



Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Row 'bw_i4' infeasible, all entries at implied bounds.
Presolve time = 0.00 sec. (0.11 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.12 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.12 ticks)

No feasible solution found
Execution time: 0.0137 seconds


**Testing using Node-Intention latency**

In [ ]:
from docplex.mp.model import Model
import time

# =========================
# Build Optimization Model
# =========================
mdl = Model("multi_intention_service_placement")

# Decision variables
x = {(s,n): mdl.binary_var(name=f"x_{s}_{n}") for s in services for n in nodes}
y = {(i,n): mdl.binary_var(name=f"y_{i}_{n}") for i in intentions for n in nodes}

# Objective: minimize weighted latency (node-based, not service-based)
mdl.minimize(
    mdl.sum(intentions[i]["weight"] *
            mdl.sum(nodes[n]["L"] * y[i,n] for n in nodes)
            for i in intentions)
)

# -------------------------
# Constraints
# -------------------------

# 1. Unique placement of each service
for s in services:
    mdl.add_constraint(mdl.sum(x[s,n] for n in nodes) == 1, ctname=f"place_{s}")

# 2. Link x and y: if a service of i is on node n, then intention i uses node n
for i in intentions:
    for s in intentions[i]["services"]:
        for n in nodes:
            mdl.add_constraint(y[i,n] >= x[s,n], ctname=f"link_{i}_{s}_{n}")

# 3. Node capacity (services only)
for n in nodes:
    for r in resources:
        mdl.add_constraint(
            mdl.sum(services[s][r] * x[s,n] for s in services) <= nodes[n][r],
            ctname=f"cap_{n}_{r}"
        )

# 4. QoS constraints
for i in intentions:
    # Latency (per node, not per service)
    mdl.add_constraint(
        mdl.sum(nodes[n]["L"] * y[i,n] for n in nodes) <= intentions[i]["QoS"]["latency"],
        ctname=f"lat_{i}"
    )
    # Bandwidth (per service, aggregated)
    mdl.add_constraint(
        mdl.sum(services[s]["BW"] * x[s,n] for s in intentions[i]["services"] for n in nodes)
        >= intentions[i]["QoS"]["bandwidth"], ctname=f"bw_{i}"
    )

# 5. Extra resources constraints (only if defined in JSON)
for i in intentions:
    if "extra_resources" in intentions[i]:
        for r, val in intentions[i]["extra_resources"].items():
            mdl.add_constraint(
                mdl.sum(services[s][r] * x[s,n] for s in intentions[i]["services"] for n in nodes)
                + val <= max(nodes[n][r] for n in nodes),
                ctname=f"extra_{i}_{r}"
            )

# =========================
# Solve
# =========================
start_time = time.time()
solution = mdl.solve(log_output=True)
end_time = time.time()
execution_time = end_time - start_time

# =========================
# Results
# =========================
if solution:
    print("\nOptimal solution found")
    print("Objective value:", solution.objective_value)
    print("Execution time:", round(execution_time, 4), "seconds")
    print("\nService placement:")
    for (s,n), var in x.items():
        if var.solution_value > 0.5:
            print(f" - Service {s} → Node {n}")
    print("\nNode usage per intention:")
    for (i,n), var in y.items():
        if var.solution_value > 0.5:
            print(f" - Intention {i} uses Node {n}")
else:
    print("\nNo feasible solution found")
    print("Execution time:", round(execution_time, 4), "seconds")


Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Row 'bw_i4' infeasible, all entries at implied bounds.
Presolve time = 0.01 sec. (0.15 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.16 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.16 ticks)

No feasible solution found
Execution time: 0.0275 seconds


**Checking why the solvor doesnt find any solution**

In [ ]:
print("\n=== Validation of intentions before solving ===")

for i in intentions:
    qos = intentions[i]["QoS"]
    services_i = intentions[i]["services"]

    # Minimum possible latency (take the smallest latency across nodes for each service)
    min_latency = sum(min(nodes[n]["L"] for n in nodes) for s in services_i)

    # Maximum possible bandwidth (sum of BW of the required services)
    total_bw_services = sum(services[s]["BW"] for s in services_i)

    print(f"\nIntention {i}:")
    print(f" - Services: {services_i}")
    print(f" - QoS required: latency <= {qos['latency']}, bandwidth >= {qos['bandwidth']}")
    print(f" - Minimum possible latency (sum) = {min_latency}")
    print(f" - Maximum possible bandwidth (sum of services) = {total_bw_services}")

    # Feasibility checks
    if min_latency > qos["latency"]:
        print("   Latency requirement is impossible with current values")
    else:
        print("   Latency requirement is feasible")

    if total_bw_services < qos["bandwidth"]:
        print("   Bandwidth requirement is impossible with current values")
    else:
        print("   Bandwidth requirement is feasible")



=== Validation of intentions before solving ===

Intention i1:
 - Services: ['s1']
 - QoS required: latency <= 60, bandwidth >= 20
 - Minimum possible latency (sum) = 30
 - Maximum possible bandwidth (sum of services) = 60
   Latency requirement is feasible
   Bandwidth requirement is feasible

Intention i2:
 - Services: ['s2', 's3']
 - QoS required: latency <= 120, bandwidth >= 150
 - Minimum possible latency (sum) = 60
 - Maximum possible bandwidth (sum of services) = 200
   Latency requirement is feasible
   Bandwidth requirement is feasible

Intention i3:
 - Services: ['s4', 's5']
 - QoS required: latency <= 130, bandwidth >= 90
 - Minimum possible latency (sum) = 60
 - Maximum possible bandwidth (sum of services) = 140
   Latency requirement is feasible
   Bandwidth requirement is feasible

Intention i4:
 - Services: ['s5', 's6']
 - QoS required: latency <= 150, bandwidth >= 120
 - Minimum possible latency (sum) = 60
 - Maximum possible bandwidth (sum of services) = 130
   Latenc

In [ ]:
print("\n=== Validation of node resource capacities ===")

for n in nodes:
    print(f"\nNode {n} capacities: {nodes[n]}")
    for s in services:
        fits = all(services[s][r] <= nodes[n][r] for r in resources)
        if fits:
            print(f" - Service {s} can fit on node {n}")
        else:
            print(f" - Service {s} cannot fit on node {n} (exceeds capacity)")



=== Validation of node resource capacities ===

Node g1 capacities: {'CPU': 2, 'MEM': 4, 'DISK': 2, 'BW': 100, 'L': 30}
 - Service s1 can fit on node g1
 - Service s2 can fit on node g1
 - Service s3 cannot fit on node g1 (exceeds capacity)
 - Service s4 cannot fit on node g1 (exceeds capacity)
 - Service s5 cannot fit on node g1 (exceeds capacity)
 - Service s6 cannot fit on node g1 (exceeds capacity)

Node g2 capacities: {'CPU': 2, 'MEM': 3, 'DISK': 1, 'BW': 80, 'L': 35}
 - Service s1 can fit on node g2
 - Service s2 cannot fit on node g2 (exceeds capacity)
 - Service s3 cannot fit on node g2 (exceeds capacity)
 - Service s4 cannot fit on node g2 (exceeds capacity)
 - Service s5 cannot fit on node g2 (exceeds capacity)
 - Service s6 cannot fit on node g2 (exceeds capacity)

Node n1 capacities: {'CPU': 4, 'MEM': 10, 'DISK': 10, 'BW': 120, 'L': 60}
 - Service s1 can fit on node n1
 - Service s2 can fit on node n1
 - Service s3 cannot fit on node n1 (exceeds capacity)
 - Service s4 can

**Testing with another JSON-**

In [ ]:
import json

with open("test3.json", "r") as f:
    data = json.load(f)

# Extract services, nodes, intentions
services = {s["id"]: s["resources"] for s in data["services"]}
nodes = {n["id"]: {**n["capacity"], "L": data["latency"][n["id"]][0]} for n in data["nodes"]}
intentions = {i["id"]: {"services": i["services"], "QoS": i["QoS"], "weight": i["weight"], "extra_resources": i.get("extra_resources", {})} for i in data["intentions"]}
resources = ["CPU","MEM","DISK","BW"]

print("✅ Data loaded successfully")
print("Services:", services.keys())
print("Nodes:", nodes.keys())
print("Intentions:", intentions.keys())

✅ Data loaded successfully
Services: dict_keys(['s1', 's2', 's3', 's4', 's5', 's6'])
Nodes: dict_keys(['g1', 'g2', 'n1', 'n2', 'n3'])
Intentions: dict_keys(['i1', 'i2', 'i3', 'i4', 'i5', 'i6'])


In [ ]:
from docplex.mp.model import Model
import time

# =========================
# Build Optimization Model
# =========================
mdl = Model("multi_intention_service_placement")

# Decision vars: x[s,n] = 1 if service s placed on node n
x = {(s,n): mdl.binary_var(name=f"x_{s}_{n}") for s in services for n in nodes}

# Objective: minimize weighted latency (sum version)
mdl.minimize(
    mdl.sum(intentions[i]["weight"] *
            mdl.sum(nodes[n]["L"] * x[s,n] for s in intentions[i]["services"] for n in nodes)
            for i in intentions)
)

# -------------------------
# Constraints
# -------------------------

# 1. Unique placement of each service
for s in services:
    mdl.add_constraint(mdl.sum(x[s,n] for n in nodes) == 1, ctname=f"place_{s}")

# 2. Node capacity (services only)
for n in nodes:
    for r in resources:
        mdl.add_constraint(
            mdl.sum(services[s][r] * x[s,n] for s in services) <= nodes[n][r],
            ctname=f"cap_{n}_{r}"
        )

# 3. QoS latency + bandwidth constraints
for i in intentions:
    # Latency constraint (sum of latencies of all services in intention i)
    mdl.add_constraint(
        mdl.sum(nodes[n]["L"] * x[s,n] for s in intentions[i]["services"] for n in nodes)
        <= intentions[i]["QoS"]["latency"], ctname=f"lat_{i}"
    )
    # Bandwidth constraint
    mdl.add_constraint(
        mdl.sum(services[s]["BW"] * x[s,n] for s in intentions[i]["services"] for n in nodes)
        >= intentions[i]["QoS"]["bandwidth"], ctname=f"bw_{i}"
    )

# 4. Extra resources constraints (only if defined in JSON)
for i in intentions:
    if "extra_resources" in intentions[i]:
        for r, val in intentions[i]["extra_resources"].items():
            mdl.add_constraint(
                mdl.sum(services[s][r] * x[s,n] for s in intentions[i]["services"] for n in nodes)
                + val <= max(nodes[n][r] for n in nodes),
                ctname=f"extra_{i}_{r}"
            )

# =========================
# Solve
# =========================
start_time = time.time()
solution = mdl.solve(log_output=True)
end_time = time.time()
execution_time = end_time - start_time

# =========================
# Results
# =========================
if solution:
    print("\nOptimal solution found")
    print("Objective value:", solution.objective_value)
    print("Execution time:", round(execution_time, 4), "seconds")
    print("\nService placement:")
    for (s,n), var in x.items():
        if var.solution_value > 0.5:
            print(f" - Service {s} → Node {n}")
else:
    print("\nNo feasible solution found")
    print("Execution time:", round(execution_time, 4), "seconds")



Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Infeasibility row 'lat_i6':  0 <= -2.
Presolve time = 0.00 sec. (0.14 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.14 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.14 ticks)

No feasible solution found
Execution time: 0.0137 seconds


In [ ]:
print("QoS latency values per intention:")
for i in intentions:
    print(i, intentions[i]["QoS"]["latency"])


QoS latency values per intention:
i1 60
i2 120
i3 130
i4 150
i5 200
i6 250


In [ ]:
from docplex.mp.model import Model
import time

# =========================
# Build Optimization Model
# =========================
mdl = Model("multi_intention_service_placement")

# Decision variables
x = {(s,n): mdl.binary_var(name=f"x_{s}_{n}") for s in services for n in nodes}
y = {(i,n): mdl.binary_var(name=f"y_{i}_{n}") for i in intentions for n in nodes}

# Objective: minimize weighted latency (node-based, not service-based)
mdl.minimize(
    mdl.sum(intentions[i]["weight"] *
            mdl.sum(nodes[n]["L"] * y[i,n] for n in nodes)
            for i in intentions)
)

# -------------------------
# Constraints
# -------------------------

# 1. Unique placement of each service
for s in services:
    mdl.add_constraint(mdl.sum(x[s,n] for n in nodes) == 1, ctname=f"place_{s}")

# 2. Link x and y: if a service of i is on node n, then intention i uses node n
for i in intentions:
    for s in intentions[i]["services"]:
        for n in nodes:
            mdl.add_constraint(y[i,n] >= x[s,n], ctname=f"link_{i}_{s}_{n}")

# 3. Node capacity (services only)
for n in nodes:
    for r in resources:
        mdl.add_constraint(
            mdl.sum(services[s][r] * x[s,n] for s in services) <= nodes[n][r],
            ctname=f"cap_{n}_{r}"
        )

# 4. QoS constraints
for i in intentions:
    # Latency (per node, not per service)
    mdl.add_constraint(
        mdl.sum(nodes[n]["L"] * y[i,n] for n in nodes) <= intentions[i]["QoS"]["latency"],
        ctname=f"lat_{i}"
    )
    # Bandwidth (per service, aggregated)
    mdl.add_constraint(
        mdl.sum(services[s]["BW"] * x[s,n] for s in intentions[i]["services"] for n in nodes)
        >= intentions[i]["QoS"]["bandwidth"], ctname=f"bw_{i}"
    )

# 5. Extra resources constraints (only if defined in JSON)
for i in intentions:
    if "extra_resources" in intentions[i]:
        for r, val in intentions[i]["extra_resources"].items():
            mdl.add_constraint(
                mdl.sum(services[s][r] * x[s,n] for s in intentions[i]["services"] for n in nodes)
                + val <= max(nodes[n][r] for n in nodes),
                ctname=f"extra_{i}_{r}"
            )

# =========================
# Solve
# =========================
start_time = time.time()
solution = mdl.solve(log_output=True)
end_time = time.time()
execution_time = end_time - start_time

# =========================
# Results
# =========================
if solution:
    print("\nOptimal solution found")
    print("Objective value:", solution.objective_value)
    print("Execution time:", round(execution_time, 4), "seconds")
    print("\nService placement:")
    for (s,n), var in x.items():
        if var.solution_value > 0.5:
            print(f" - Service {s} → Node {n}")
    print("\nNode usage per intention:")
    for (i,n), var in y.items():
        if var.solution_value > 0.5:
            print(f" - Intention {i} uses Node {n}")
else:
    print("\nNo feasible solution found")
    print("Execution time:", round(execution_time, 4), "seconds")


Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 43 rows and 14 columns.
MIP Presolve modified 159 coefficients.
Aggregator did 8 substitutions.
Reduced MIP has 67 rows, 38 columns, and 180 nonzeros.
Reduced MIP has 38 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.33 ticks)
Found incumbent of value 6930.000000 after 0.01 sec. (0.50 ticks)
Probing time = 0.00 sec. (0.03 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 67 rows, 38 columns, and 180 nonzeros.
Reduced MIP has 38 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.13 ticks)
Probing time = 0.00 sec. (0.03 ticks)
Clique table members: 61.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root relaxation solution time = 0.02 sec. (0.21 ticks)

        Nodes   

**Test with Complex Data **

In [ ]:
import json

with open("New_DataSet.txt", "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract services, nodes, intentions
services = {s["id"]: s["resources"] for s in data["services"]}
nodes = {
    n["id"]: {**n["capacity"], "L": data["latency"][n["id"]][0]}
    for n in data["nodes"]
}
intentions = {
    i["id"]: {
        "services": i["services"],
        "QoS": i["QoS"],
        "weight": i["weight"],
        "extra_resources": i.get("extra_resources", {})
    }
    for i in data["intentions"]
}
resources = ["CPU", "MEM", "DISK", "BW"]

print(" Data loaded successfully")
print("Services:", services.keys())
print("Nodes:", nodes.keys())
print("Intentions:", intentions.keys())


 Data loaded successfully
Services: dict_keys(['s1', 's2', 's3', 's4', 's5', 's6'])
Nodes: dict_keys(['g1', 'g2', 'g3', 'g4', 'g5', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15'])
Intentions: dict_keys(['i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7', 'i9', 'i10', 'i11', 'i12', 'i13', 'i14', 'i15', 'i16', 'i17', 'i18', 'i19', 'i20', 'i21', 'i22', 'i23', 'i24', 'i25', 'i26', 'i27', 'i28', 'i29', 'i30', 'i32', 'i33', 'i34', 'i35', 'i36', 'i37', 'i38', 'i39', 'i40', 'i41', 'i42', 'i43', 'i44', 'i45', 'i46', 'i47', 'i48', 'i49', 'i50'])


In [ ]:
from docplex.mp.model import Model
import time

# =========================
# Build Optimization Model
# =========================
mdl = Model("multi_intention_service_placement")

# Decision variables
x = {(s,n): mdl.binary_var(name=f"x_{s}_{n}") for s in services for n in nodes}
y = {(i,n): mdl.binary_var(name=f"y_{i}_{n}") for i in intentions for n in nodes}

# Objective: minimize weighted latency (node-based, not service-based)
mdl.minimize(
    mdl.sum(intentions[i]["weight"] *
            mdl.sum(nodes[n]["L"] * y[i,n] for n in nodes)
            for i in intentions)
)

# -------------------------
# Constraints
# -------------------------

# 1. Unique placement of each service
for s in services:
    mdl.add_constraint(mdl.sum(x[s,n] for n in nodes) == 1, ctname=f"place_{s}")

# 2. Link x and y: if a service of i is on node n, then intention i uses node n
for i in intentions:
    for s in intentions[i]["services"]:
        for n in nodes:
            mdl.add_constraint(y[i,n] >= x[s,n], ctname=f"link_{i}_{s}_{n}")

# 3. Node capacity (services only)
for n in nodes:
    for r in resources:
        mdl.add_constraint(
            mdl.sum(services[s][r] * x[s,n] for s in services) <= nodes[n][r],
            ctname=f"cap_{n}_{r}"
        )

# 4. QoS constraints
for i in intentions:
    # Latency (per node, not per service)
    mdl.add_constraint(
        mdl.sum(nodes[n]["L"] * y[i,n] for n in nodes) <= intentions[i]["QoS"]["latency"],
        ctname=f"lat_{i}"
    )
    # Bandwidth (per service, aggregated)
    mdl.add_constraint(
        mdl.sum(services[s]["BW"] * x[s,n] for s in intentions[i]["services"] for n in nodes)
        >= intentions[i]["QoS"]["bandwidth"], ctname=f"bw_{i}"
    )

# 5. Extra resources constraints (only if defined in JSON)
for i in intentions:
    if "extra_resources" in intentions[i]:
        for r, val in intentions[i]["extra_resources"].items():
            mdl.add_constraint(
                mdl.sum(services[s][r] * x[s,n] for s in intentions[i]["services"] for n in nodes)
                + val <= max(nodes[n][r] for n in nodes),
                ctname=f"extra_{i}_{r}"
            )

# =========================
# Solve
# =========================
start_time = time.time()
solution = mdl.solve(log_output=True)
end_time = time.time()
execution_time = end_time - start_time

# =========================
# Results
# =========================
if solution:
    print("\nOptimal solution found")
    print("Objective value:", solution.objective_value)
    print("Execution time:", round(execution_time, 4), "seconds")
    print("\nService placement:")
    for (s,n), var in x.items():
        if var.solution_value > 0.5:
            print(f" - Service {s} → Node {n}")
    print("\nNode usage per intention:")
    for (i,n), var in y.items():
        if var.solution_value > 0.5:
            print(f" - Intention {i} uses Node {n}")
else:
    print("\nNo feasible solution found")
    print("Execution time:", round(execution_time, 4), "seconds")


Using size restricted mode (Could not find directory for cpxchecklic).
CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.


DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 1120 vars, 2286 consts, CPLEX code=1016

*** **`texte en gras`**Another Solver: ***

In [ ]:
!pip install pulp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 149.2 MB/s eta 0:00:00


In [ ]:
import time
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, PULP_CBC_CMD

# =========================
# Build Optimization Model
# =========================
prob = LpProblem("multi_intention_service_placement", LpMinimize)

# Decision variables
x = {(s,n): LpVariable(f"x_{s}_{n}", cat="Binary") for s in services for n in nodes}
y = {(i,n): LpVariable(f"y_{i}_{n}", cat="Binary") for i in intentions for n in nodes}

# Objective: minimize weighted latency
prob += lpSum(
    intentions[i]["weight"] *
    lpSum(nodes[n]["L"] * y[i,n] for n in nodes)
    for i in intentions
)

# -------------------------
# Constraints
# -------------------------

# 1. Unique placement of each service
for s in services:
    prob += lpSum(x[s,n] for n in nodes) == 1, f"place_{s}"

# 2. Link x and y: if a service of i is on node n, then intention i uses node n
for i in intentions:
    for s in intentions[i]["services"]:
        for n in nodes:
            prob += y[i,n] >= x[s,n], f"link_{i}_{s}_{n}"

# 3. Node capacity (services only)
for n in nodes:
    for r in resources:
        prob += (
            lpSum(services[s][r] * x[s,n] for s in services) <= nodes[n][r],
            f"cap_{n}_{r}"
        )

# 4. QoS constraints
for i in intentions:
    # Latency
    prob += (
        lpSum(nodes[n]["L"] * y[i,n] for n in nodes) <= intentions[i]["QoS"]["latency"],
        f"lat_{i}"
    )
    # Bandwidth
    prob += (
        lpSum(services[s]["BW"] * x[s,n] for s in intentions[i]["services"] for n in nodes)
        >= intentions[i]["QoS"]["bandwidth"],
        f"bw_{i}"
    )

# 5. Extra resources constraints (if defined)
for i in intentions:
    if "extra_resources" in intentions[i]:
        for r, val in intentions[i]["extra_resources"].items():
            prob += (
                lpSum(services[s][r] * x[s,n] for s in intentions[i]["services"] for n in nodes)
                + val <= max(nodes[n][r] for n in nodes),
                f"extra_{i}_{r}"
            )

# =========================
# Solve
# =========================
start_time = time.time()
prob.solve(PULP_CBC_CMD(msg=1))  # CBC solver
end_time = time.time()
execution_time = end_time - start_time

# =========================
# Results
# =========================
status = prob.status
if status == 1:  # Optimal
    print("\n Optimal solution found")
    print("Objective value:", prob.objective.value())
    print("Execution time:", round(execution_time, 4), "seconds")
    print("\nService placement:")
    for (s,n), var in x.items():
        if var.value() > 0.5:
            print(f" - Service {s} → Node {n}")
    print("\nNode usage per intention:")
    for (i,n), var in y.items():
        if var.value() > 0.5:
            print(f" - Intention {i} uses Node {n}")
else:
    print("\n No feasible solution found")
    print("Execution time:", round(execution_time, 4), "seconds")



 Optimal solution found
Objective value: 23060.0
Execution time: 0.4965 seconds

Service placement:
 - Service s1 → Node g1
 - Service s2 → Node n12
 - Service s3 → Node n9
 - Service s4 → Node n9
 - Service s5 → Node n12
 - Service s6 → Node n12

Node usage per intention:
 - Intention i1 uses Node g1
 - Intention i2 uses Node n12
 - Intention i3 uses Node n9
 - Intention i4 uses Node n9
 - Intention i4 uses Node n12
 - Intention i5 uses Node n12
 - Intention i6 uses Node n12
 - Intention i7 uses Node n9
 - Intention i9 uses Node n9
 - Intention i10 uses Node n12
 - Intention i11 uses Node n12
 - Intention i12 uses Node n9
 - Intention i13 uses Node n12
 - Intention i14 uses Node n9
 - Intention i14 uses Node n12
 - Intention i15 uses Node g1
 - Intention i15 uses Node n9
 - Intention i15 uses Node n12
 - Intention i16 uses Node n12
 - Intention i17 uses Node n9
 - Intention i17 uses Node n12
 - Intention i18 uses Node n12
 - Intention i19 uses Node n9
 - Intention i20 uses Node n12
 

In [ ]:
# =========================
# Feasibility Diagnostic
# =========================

print("=== Checking feasibility of dataset ===")

# 1. Check if each service can fit in at least one node
for s in services:
    fits = [
        n for n in nodes
        if all(services[s][r] <= nodes[n][r] for r in resources)
    ]
    if not fits:
        print(f"⚠️ Service {s} cannot fit into ANY node (too many resources required).")
    else:
        print(f"Service {s} can fit into nodes: {fits}")

# 2. Check latency constraints
for i in intentions:
    min_latency = min(nodes[n]["L"] for n in nodes)
    if intentions[i]["QoS"]["latency"] < min_latency:
        print(f"⚠️ Intention {i} requires latency {intentions[i]['QoS']['latency']} "
              f"but the minimum available is {min_latency} → impossible.")

# 3. Check bandwidth constraints
for i in intentions:
    required_bw = intentions[i]["QoS"]["bandwidth"]
    total_service_bw = sum(services[s]["BW"] for s in intentions[i]["services"])
    max_node_bw = max(nodes[n]["BW"] for n in nodes)

    if required_bw > total_service_bw:
        print(f"⚠️ Intention {i} requires bandwidth {required_bw}, "
              f"but its services provide only {total_service_bw}.")
    elif required_bw > max_node_bw:
        print(f"⚠️ Intention {i} requires bandwidth {required_bw}, "
              f"but no node can provide more than {max_node_bw}.")

# 4. Check extra resources (if defined)
for i in intentions:
    if "extra_resources" in intentions[i]:
        for r, val in intentions[i]["extra_resources"].items():
            max_available = max(nodes[n][r] for n in nodes)
            total_needed = sum(services[s][r] for s in intentions[i]["services"]) + val
            if total_needed > max_available:
                print(f"⚠️ Intention {i} with extra {r}={val} requires {total_needed}, "
                      f"but max available on any node is {max_available}.")

print("=== Feasibility check complete ===")


=== Checking feasibility of dataset ===
Service s1 can fit into nodes: ['g1', 'g2', 'g3', 'g4', 'g5', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15']
Service s2 can fit into nodes: ['g1', 'g3', 'g4', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15']
Service s3 can fit into nodes: ['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15']
Service s4 can fit into nodes: ['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15']
Service s5 can fit into nodes: ['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15']
Service s6 can fit into nodes: ['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15']
⚠️ Intention i8 requires bandwidth 60, but its services provide only 40.
⚠️ Intention i31 requires bandwidth 60, but its services provide only 4

**Test with the new Dataset**

In [ ]:
import time
import json
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, PULP_CBC_CMD

# =========================================
# 1) Load test3.json (already in Colab)
# =========================================

JSON_FILE = "/content/sample_data/test3.json"

with open(JSON_FILE, "r") as f:
    data = json.load(f)

print("test3.json successfully loaded")

# =========================================
# 2) Data preparation
# =========================================

# Resources (CPU, MEM, DISK, BW)
resources = list(data["services"][0]["resources"].keys())

# ---------- Services ----------

services = {}
for s in data["services"]:
    sid = s["id"]
    services[sid] = s["resources"].copy()
    services[sid]["name"] = s["name"]

# ---------- Nodes ----------

nodes = {}
for node in data["nodes"]:
    nid = node["id"]
    nodes[nid] = node["capacity"].copy()


    lat_list = data["latency"][nid]

    nodes[nid]["L"] = sum(lat_list) / len(lat_list)

# ---------- Intentions ----------
intentions = {}
for inten in data["intentions"]:
    iid = inten["id"]
    intentions[iid] = {
        "services": inten["services"],
        "QoS": inten["QoS"],
        "weight": inten["weight"],
        "description": inten["description"],
    }
    if "extra_resources" in inten:
        intentions[iid]["extra_resources"] = inten["extra_resources"]

# =========================================
# 3) Optimization model construction
# =========================================

prob = LpProblem("multi_intention_service_placement", LpMinimize)

# Decision variables
x = {(s, n): LpVariable(f"x_{s}_{n}", cat="Binary")
     for s in services for n in nodes}

y = {(i, n): LpVariable(f"y_{i}_{n}", cat="Binary")
     for i in intentions for n in nodes}

# -----------------------------------------
# Objective: Minimize weighted latency
# -----------------------------------------
prob += lpSum(
    intentions[i]["weight"] *
    lpSum(nodes[n]["L"] * y[i, n] for n in nodes)
    for i in intentions
)

# =========================================
# 4) Constraints
# =========================================

# 1. Each service must be placed on exactly one node
for s in services:
    prob += lpSum(x[s, n] for n in nodes) == 1, f"place_{s}"

# 2. Link between x and y:
# if a service of intention i is placed on node n,
# then intention i uses node n
for i in intentions:
    for s in intentions[i]["services"]:
        for n in nodes:
            prob += y[i, n] >= x[s, n], f"link_{i}_{s}_{n}"

# 3. Node capacity constraints for all resources
for n in nodes:
    for r in resources:
        prob += (
            lpSum(services[s][r] * x[s, n] for s in services)
            <= nodes[n][r]
        ), f"cap_{n}_{r}"

# 4. QoS constraints for each intention
for i in intentions:
    # Maximum latency
    prob += (
        lpSum(nodes[n]["L"] * y[i, n] for n in nodes)
        <= intentions[i]["QoS"]["latency"]
    ), f"lat_{i}"

    # Minimum bandwidth
    prob += (
        lpSum(
            services[s]["BW"] * x[s, n]
            for s in intentions[i]["services"]
            for n in nodes
        )
        >= intentions[i]["QoS"]["bandwidth"]
    ), f"bw_{i}"

# 5. Additional resource constraints (if defined)
for i in intentions:
    if "extra_resources" in intentions[i]:
        for r, val in intentions[i]["extra_resources"].items():
            prob += (
                lpSum(
                    services[s][r] * x[s, n]
                    for s in intentions[i]["services"]
                    for n in nodes
                ) + val
                <= max(nodes[n][r] for n in nodes)
            ), f"extra_{i}_{r}"

# =========================================
# 5) Solve the optimization problem
# =========================================

start_time = time.time()
prob.solve(PULP_CBC_CMD(msg=1))  # CBC solver
end_time = time.time()
execution_time = end_time - start_time

# =========================================
# 6) Results
# =========================================

status = prob.status
if status == 1:  # Optimal
    print("\n Optimal solution found")
    print("Objective value:", prob.objective.value())
    print("Execution time:", round(execution_time, 4), "seconds")

    print("\n Service placement:")
    for (s, n), var in x.items():
        if var.value() > 0.5:
            print(f" - Service {s} ({services[s]['name']}) → Node {n}")

    print("\n Node usage per intention:")
    for (i, n), var in y.items():
        if var.value() > 0.5:
            print(f" - Intention {i} ({intentions[i]['description']}) uses Node {n}")
else:
    print("\n No feasible solution found")
    print("Execution time:", round(execution_time, 4), "seconds")


test3.json successfully loaded

 Optimal solution found
Objective value: 5618.333333333333
Execution time: 0.2183 seconds

 Service placement:
 - Service s1 (Voice_Converter) → Node g1
 - Service s2 (Protocol_Converter) → Node n3
 - Service s3 (AR_Generator) → Node n3
 - Service s4 (Content_Displayer) → Node n2
 - Service s5 (Error_Detector) → Node n2
 - Service s6 (Anomaly_Analyzer) → Node n2

 Node usage per intention:
 - Intention i1 (Quick voice conversion) uses Node g1
 - Intention i2 (AR generation with protocol conversion) uses Node n3
 - Intention i3 (Content display with error detection) uses Node n2
 - Intention i4 (Full anomaly analysis) uses Node n2
 - Intention i5 (AR pipeline) uses Node g1
 - Intention i5 (AR pipeline) uses Node n2
 - Intention i5 (AR pipeline) uses Node n3
 - Intention i6 (Heavy workload (all services)) uses Node g1
 - Intention i6 (Heavy workload (all services)) uses Node n2
 - Intention i6 (Heavy workload (all services)) uses Node n3
